# Mental Health FAQ Chatbot with Gemini API

This notebook sets up a mental health chatbot using Google's Gemini API and the Mental Health FAQ dataset.


## 1. Install Required Libraries


In [ ]:
!pip install google-generativeai pandas python-dotenv


## 2. Import Libraries and Setup


In [ ]:
import google.generativeai as genai
import pandas as pd
import os
from dotenv import load_dotenv
import json

# Configure Gemini API
# Replace with your actual API key or set it as environment variable
GEMINI_API_KEY = "YOUR_GEMINI_API_KEY_HERE"  # Get from: https://makersuite.google.com/app/apikey

genai.configure(api_key=GEMINI_API_KEY)


## 3. Upload and Load FAQ Data


In [ ]:
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Load the CSV file
df = pd.read_csv('Mental_Health_FAQ.csv')
print(f"Loaded {len(df)} FAQ entries")
print(df.head())


## 4. Prepare FAQ Data for Context


In [ ]:
# Create a formatted context string from FAQ data
def create_faq_context(df):
    """Create a formatted context string from the FAQ dataframe"""
    context_parts = []
    for idx, row in df.iterrows():
        q = str(row['Questions']).strip()
        a = str(row['Answers']).strip()
        context_parts.append(f"Q: {q}\nA: {a}\n")
    return "\n".join(context_parts)

# Create the context
faq_context = create_faq_context(df)
print(f"Context length: {len(faq_context)} characters")
print(f"Number of FAQ entries: {len(df)}")


## 5. Save Context for Flask App


In [ ]:
# Save FAQ data as JSON for easy loading in Flask
faq_data = []
for idx, row in df.iterrows():
    faq_data.append({
        'question_id': str(row['Question_ID']),
        'question': str(row['Questions']).strip(),
        'answer': str(row['Answers']).strip()
    })

with open('faq_data.json', 'w', encoding='utf-8') as f:
    json.dump(faq_data, f, ensure_ascii=False, indent=2)

print(f"Saved {len(faq_data)} FAQ entries to faq_data.json")

# Download the file
files.download('faq_data.json')


## 6. Test the Chatbot


In [ ]:
# Create system prompt with FAQ context
system_prompt = f"""You are a helpful and empathetic mental health assistant chatbot. 
Your role is to provide accurate, supportive, and compassionate information about mental health based on the following FAQ database.

FAQ Database:
{faq_context[:5000]}...

Instructions:
1. Use the FAQ database above to answer user questions accurately
2. If a question matches or is similar to a FAQ entry, provide that answer
3. If the question is not directly in the FAQ, use your knowledge to provide helpful, empathetic responses
4. Always be supportive, non-judgmental, and encourage professional help when appropriate
5. If someone is in crisis, encourage them to seek immediate professional help
6. Keep responses clear, concise, and easy to understand

Remember: You are not a replacement for professional mental health care. Always encourage users to consult with qualified mental health professionals for diagnosis and treatment.
"""

# Initialize the model
model = genai.GenerativeModel('gemini-pro')

# Test query
test_question = "What does it mean to have a mental illness?"

full_prompt = f"{system_prompt}\n\nUser Question: {test_question}"

response = model.generate_content(full_prompt)
print("Question:", test_question)
print("\nAnswer:")
print(response.text)
